In [1]:
import os

import torch
import numpy as np
from IPython.utils import io
import random
from evaluate import load
from torchvision import transforms
from tqdm import tqdm
from undecorated import undecorated
from types import MethodType
from torch.utils.data import DataLoader
from torchmetrics.multimodal import CLIPScore
from peft import PeftModel, PeftModelForCausalLM, prepare_model_for_int8_training, LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoFeatureExtractor

from generate_model_seq_llama import SeqGen
from general_dataset import GeneralDataset
from agi_utils import *
from combine_model_seq import SeqCombine

/common/home/yg334/anaconda3/envs/vicuna/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /common/home/yg334/anaconda3/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 111
CUDA SETUP: Loading binary /common/home/yg334/anaconda3/envs/vicuna/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda111.so...


In [2]:
"""
assign openagi data path 
"""
data_path = "YOUR_DATA_PATH"

task_discriptions = txt_loader("./task_description.txt")
training_task_idx = [7,20,30,40,50,60]
training_dataloaders = []
for i in tqdm(training_task_idx):
    dataset = GeneralDataset(i,data_path)
    dataloader = DataLoader(dataset, batch_size=5)
    training_dataloaders.append(dataloader)

test_task_idx = [2,3,10,15,20,35,45,55,65,70,70,90,106,107]
test_dataloaders = []
for i in tqdm(test_task_idx):
    dataset = GeneralDataset(i, data_path)
    dataloader = DataLoader(dataset, batch_size=5)
    test_dataloaders.append(dataloader)
    
test_tasks = [task_discriptions[i].strip() for i in test_task_idx]
training_tasks = [task_discriptions[i].strip() for i in training_task_idx]

100%|███████████████████████████████████████████| 14/14 [00:03<00:00,  3.93it/s]


In [3]:
base_model = "eachadea/vicuna-7b-1.1"
load_8bit = True

max_memory_mapping = {
    0: "24GB",
    1: "24GB",
    2: "24GB",
    3: "24GB",
    4: "0GB",
    5: "0GB",
    6: "0GB",
    7: "0GB",
}


tokenizer = AutoTokenizer.from_pretrained(
    "eachadea/vicuna-7b-1.1",
)
tokenizer.add_special_tokens({'pad_token': '<pad>'})

model = AutoModelForCausalLM.from_pretrained(
    "eachadea/vicuna-7b-1.1",
    device_map="auto",
    max_memory=max_memory_mapping,
)

generate_with_grad = undecorated(model.generate)
model.generate_with_grad = MethodType(generate_with_grad, model)


lora_weights = "YOUR_LORA_WEIGHTS"

model = PeftModelForCausalLM.from_pretrained(
                                            model,
                                            lora_weights,
                                            torch_dtype=torch.float16,
                                            is_trainable=True
                                            )


model.print_trainable_parameters()


seqGen = SeqGen(model, tokenizer)

Loading checkpoint shards: 100%|██████████████████| 2/2 [02:22<00:00, 71.48s/it]


trainable params: 4194304 || all params: 6742609920 || trainable%: 0.06220594176090199


In [4]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--seed", type=int, default=42)
parser.add_argument("--epochs", type=int, default=5)
parser.add_argument("--batch_size", type=int, default=5)
parser.add_argument("--num_seq", type=int, default=1)
parser.add_argument("--learning_rate", type=float, default=1e-5)
parser.add_argument("--epsilon", type=float, default=0.2)
parser.add_argument("--decay_rate", type=float, default=0.9)
parser.add_argument("--weight_decay", type=float, default=1e-6)
parser.add_argument("--accumulate_steps", type=int, default=1)
parser.add_argument("--warm_up_proportion", type=float, default=0.1)

args = parser.parse_args("")


import openai
openai.api_key = "YOUR_OPENAPI_KEY"

def generate_module_list_with_gpt(generated_module_seq):
    todo_prompt = "You are a key phrase extractor who is able to extract potential module names from the given context. You have already known all the module names in the full module list. The full module list is: [Image Classification, Colorization, Object Detection, Image Deblurring, Image Denoising, Image Super Resolution, Image Captioning, Text to Image Generation, Visual Question Answering, Sentiment Analysis, Question Answering, Text Summarization, Machine Translation]. Given the following context: '{}'. Please extract a module sequence from this context and remove module names which do not exist in the full module list from this sequence. Output the module sequence after filtering as the format of 'module: module1, module: module2, module: module3, etc...'. "
    prompt = todo_prompt.format(generated_module_seq)

    completion = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )

    content = completion.choices[0].message["content"]
    
    # print(content)
    
    content = content.split("module: ")[1:]
    
    result = ""
    for c in content:
        result += c
    
    # result = result[:-1] if len(result) > 0 else result
    
    return result

In [5]:
"""
Loading Evaluation Metrics
"""
from sentence_transformers import SentenceTransformer, util
sentence_model = SentenceTransformer('all-MiniLM-L6-v2', device="cpu")


clip_score = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")


# Load a pre-trained Vision Transformer model and its feature extractor
vit_ckpt = "nateraw/vit-base-beans"
vit = AutoModel.from_pretrained(vit_ckpt)
vit.eval()
vit_extractor = AutoFeatureExtractor.from_pretrained(vit_ckpt)

f = transforms.ToPILImage()
bertscore = load("bertscore")

# device_list = ["cuda:1","cuda:2","cuda:3","cuda:4","cuda:5","cuda:7","cpu"]
device_list = ["cuda:4","cpu"]
seqCombination = SeqCombine(device_list)

from utils import construct_optimizer
optimizer, scheduler = construct_optimizer(args, model, args.epochs)

Some weights of the model checkpoint at nateraw/vit-base-beans were not used when initializing ViTModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTModel were not initialized from the model checkpoint at nateraw/vit-base-beans and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
rewards = []
clips = []
berts = []
similairies = []

module_length = 4
num_beams = 1
num_return_sequences = 1

eval_device = "cuda:5"

epsilon = args.epsilon
decay_rate = args.decay_rate
module_length = 10

for e in range(args.epochs):
    baseline = 0
    rewards = []

    for i, task_description in enumerate(tqdm(training_tasks)):
        # if i == 1:
        #     break

        print(task_description)
        optimizer.zero_grad()
        task_rewards = []
        input_s = ["### Human: "+task_description+"\n### Assistant: "]
        input_ids = tokenizer.batch_encode_plus(
            input_s, padding="longest", return_tensors="pt"
        )["input_ids"].to(eval_device)

        generated_module_seq, log_prob = seqGen.generate_sequence(input_ids,\
                                                                   module_length=4, \
                                                                   num_beams=20, \
                                                                   num_return_sequences=20,\
                                                                   top_k=40,\
                                                                   top_p=0.75,\
                                                                   temperature=0.2,\
                                                                   num_beam_groups=1,\
                                                                   max_length=50)
        
    
        if random.random() >= epsilon:
            action = torch.argmax(torch.stack(log_prob).detach())
        else:
            action = torch.distributions.Categorical(torch.stack(log_prob).detach()).sample()

        # decrease epsilon by the decay rate after each step
        epsilon *= decay_rate

    
        vicuna_steps = generated_module_seq[action].strip()[:-1].split(",")
        module_list = match_module_seq(vicuna_steps, sentence_model)
        # print(output_sequence)
        print(module_list)
        


        # if len(module_list) >= 1 and module_seq_filter(module_list, test_task_idx[i]):
        if len(module_list) >= 1 and whole_module_seq_filter(module_list, test_task_idx[i]):
            seqCombination.construct_module_seq(module_list)

            for idx, batch in enumerate(test_dataloaders[i]):
                inputs = list(batch['input'][0])
                # print("Inputs: ", inputs)
                try:
                    predictions = seqCombination.run_module_seq(inputs)
                except:
                    ave_task_reward = 0
                    break

                if 0 <= test_task_idx[i] <= 14:
                    outputs = list(batch['output'][0])
                    dist = image_similarity(predictions, outputs, vit, vit_extractor)
                    task_rewards.append(dist / 100)
                elif 15 <= test_task_idx[i] <= 104 or 107 <= test_task_idx[i]:
                    outputs = list(batch['output'][0])
                    f1 = np.mean(txt_eval(predictions, outputs, bertscore, device=eval_device))

                    task_rewards.append(f1)
                else:
                    score = clip_score(predictions, inputs)
                    task_rewards.append(score.detach()/100)

            ave_task_reward = np.mean(task_rewards)  
            rewards.append(ave_task_reward)
            seqCombination.close_module_seq()

        else:
            rewards.append(-1)   

        avg_reward = np.mean(rewards)
        # print("Average reward: " + str(avg_reward))
        loss = -log_prob[action] * (avg_reward - baseline)
        print("Loss: "+ str(loss.item()))
        loss.backward()
        optimizer.step()
        scheduler.step()
        # baseline = avg_reward

print("Finished training!")    

  0%|                                                     | 0/6 [00:00<?, ?it/s]

Given grayscale image, how to return the regular image step by step?
Image Denoising, Image Deblurring, Image Super Resolution, Object Detection
Loss: -22.35538101196289


 17%|███████▎                                    | 1/6 [03:07<15:39, 187.83s/it]

Given low-resolutioned noisy blurry grayscale image, how to return the object names in English step by step?
Image Super Resolution, Image Deblurring
Loss: -1.417036771774292


 33%|██████████████▋                             | 2/6 [06:21<12:44, 191.23s/it]

...

 83%|████████████████████████████████████▋       | 5/6 [08:02<01:58, 118.99s/it]

Given grayscale image, how to return the caption in English step by step?
Image Denoising, Image Deblurring, Image Super Resolution, Object Detection


Loss: -0.9058508276939392


100%|████████████████████████████████████████████| 6/6 [10:28<00:00, 104.69s/it]

Finished training!


In [10]:
rewards = []
clips = []
berts = []
similairies = []

module_length = 10
num_beams = 1
num_return_sequences = 1

eval_device = "cuda:5"
ave_results = np.array([0.,0.,0.,0.])

for t in range(5):
    for i, task_description in enumerate(tqdm(test_tasks)):
        print(task_description)
        task_rewards = []
        with torch.no_grad():
            input_s = ["### Human: "+augment_prompt(task_description,0)+"\n### Assistant: "]
            input_ids = tokenizer.batch_encode_plus(
                input_s, padding="longest", return_tensors="pt"
            )["input_ids"].to(eval_device)
            output = model.generate(
                input_ids=input_ids,
                max_length=2048, return_dict_in_generate=True, output_scores=True, output_hidden_states=True,repetition_penalty=1.25
            )
        generated_seq = tokenizer.decode(
            output["sequences"][0], skip_special_tokens=True, temperature=0, top_p=0.8, repetition_penalty=1.25
        )[len(input_s[0]):]
        print(generated_seq)
        vicuna_steps = generate_module_list_with_gpt(generated_seq).split(",")
        module_list = match_module_seq(vicuna_steps, sentence_model)
        print(module_list)



        # if len(module_list) >= 1 and module_seq_filter(module_list, test_task_idx[i]):
        if len(module_list) >= 1 and whole_module_seq_filter(module_list, test_task_idx[i]):
            seqCombination.construct_module_seq(module_list)

            for idx, batch in enumerate(test_dataloaders[i]):
                inputs = list(batch['input'][0])
                # print("Inputs: ", inputs)
                try:
                    predictions = seqCombination.run_module_seq(inputs)
                except:
                    ave_task_reward = 0
                    break

                if 0 <= test_task_idx[i] <= 14:
                    outputs = list(batch['output'][0])
                    dist = image_similarity(predictions, outputs, vit, vit_extractor)
                    task_rewards.append(dist / 100)
                elif 15 <= test_task_idx[i] <= 104 or 107 <= test_task_idx[i]:
                    outputs = list(batch['output'][0])
                    f1 = np.mean(txt_eval(predictions, outputs, bertscore, device=eval_device))

                    task_rewards.append(f1)
                else:
                    score = clip_score(predictions, inputs)
                    task_rewards.append(score.detach()/100)

            ave_task_reward = np.mean(task_rewards)    
            seqCombination.close_module_seq()

        else:
            ave_task_reward = 0

        print(ave_task_reward)
        if 0 <= test_task_idx[i] <= 14:
            similairies.append(ave_task_reward)
        elif 15 <= test_task_idx[i] <= 104 or 107 <= test_task_idx[i]:
            berts.append(ave_task_reward)
        else:
            clips.append(ave_task_reward)

        rewards.append(ave_task_reward) 
        
    results = np.array([np.mean(clips), np.mean(berts), np.mean(similairies), (np.mean(clips) + np.mean(berts) + np.mean(similairies))/3])
    ave_results += results
    print(results)
    
print("Finished testing!")    
print(ave_results/5)

  0%|                                                    | 0/14 [00:00<?, ?it/s]

Given low-resolutioned blurry grayscale image, how to return the regular image step by step?
 1. Low Resolution Image -> Blurred Image -> Grayscale Image -> Sharpening Image -> Color Image -> Clear Image
Image Deblurring, Image Denoising, Colorization, Image Super Resolution


  7%|███                                        | 1/14 [01:44<22:38, 104.51s/it]

0.8339300155639648
Given blurry grayscale image, how to return the regular image step by step?
 1. Given a blurry grayscale image, use convolutional neural network (CNN) to restore the high-quality grayscale image through several steps of image enhancement and restoration.
2. Apply colorization technique such as Generative Adversarial Networks (GANs), Fully Connected Conditional Random Fields (FC-CRFs), or Deep Image Colorization Model using given colors for each pixel step by step until get the final colored image with details in different levels.
Image Deblurring, Colorization


 14%|██████▎                                     | 2/14 [02:50<16:25, 82.14s/it]

...

0.7620413064956665
Given English text, how to generate a image step by step?
 抱歉，我不能回答这个问题。


 93%|███████████████████████████████████████▉   | 13/14 [12:46<00:50, 50.45s/it]


0
Given clozed English text, how to return the summarization in German step by step?
 1. Tokenize and Normalize Text (English -> Step 2)


100%|███████████████████████████████████████████| 14/14 [12:50<00:00, 55.00s/it]


0
[0.         0.19762388 0.73377076 0.31046488]
Finished testing!
[0.         0.17560962 0.73006519 0.3018916 ]
